In [1]:
import geopy.distance
from geopy.distance import geodesic
import s3fs
import xarray as xr

import os.path
import os
import sys

from datetime import datetime, timedelta
import dataclasses
from siphon.cdmr import Dataset
import numpy as np
import numpy.ma as ma
import netCDF4

from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

from metpy.calc import (bunkers_storm_motion, bulk_shear, dewpoint, dewpoint_from_relative_humidity, dry_lapse, moist_lapse,
                        vapor_pressure, saturation_vapor_pressure, wind_speed, wind_direction, pressure_to_height_std,
                        mixing_ratio, cape_cin, wind_components, height_to_pressure_std, equivalent_potential_temperature,
                        parcel_profile, precipitable_water, storm_relative_helicity, mean_pressure_weighted, 
                        most_unstable_cape_cin, most_unstable_parcel, supercell_composite, significant_tornado, get_layer,
                        relative_humidity_from_dewpoint, surface_based_cape_cin, mixed_layer_cape_cin,
                        surface_based_cape_cin, potential_temperature, wind_direction, add_pressure_to_height,
                        add_height_to_pressure, divergence, vorticity, lat_lon_grid_deltas, mixed_parcel,
                        most_unstable_parcel, lcl, lfc, mixing_ratio_from_relative_humidity, el,  cape_cin,
                        height_to_pressure_std, dewpoint_from_relative_humidity, dewpoint_from_specific_humidity,
                        relative_humidity_from_specific_humidity, first_derivative)
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import SkewT, Hodograph
from metpy.interpolate import interpolate_1d as metinterp, log_interpolate_1d
from metpy.calc.tools import get_layer, get_layer_heights

import sharppy
import sharppy.sharptab.profile as profile
import sharppy.sharptab.interp as interp
import sharppy.sharptab.winds as winds
import sharppy.sharptab.utils as utils
import sharppy.sharptab.params as params
import sharppy.sharptab.thermo as thermo

from ecape.calc import calc_ecape

import csv
import pandas as pd
import math
import time

D:\Anaconda3\envs\THESIS\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#https://citylikeamradio.github.io/ecape/html/index.html
# SPDX-FileCopyrightText: 2023-present Robert Capella <bob.capella@gmail.com>
# SPDX-License-Identifier: MIT
"""Calculate the entraining CAPE (ECAPE) of a parcel"""
from typing import Callable, Tuple

import pint
from metpy.constants import dry_air_spec_heat_press, earth_gravity
from metpy.units import check_units, units

PintList = np.typing.NDArray[pint.Quantity]

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

@check_units("[pressure]", "[temperature]", "[temperature]")
def _get_parcel_profile(
    pressure: PintList, temperature: PintList, dew_point_temperature: PintList, parcel_func: Callable = None
) -> PintList:
    """
    Retrieve a parcel's temperature profile.

    Args:
        pressure:
            Total atmospheric pressure
        temperature:
            Air temperature
        dew_point_temperature:
            Dew point temperature
        parcel_func:
            parcel profile retrieval callable via MetPy

    Returns:
        parcel_profile

    """

    # if surface-based, skip this process, None is default for lfc, el MetPy calcs
    if parcel_func:
        # calculate the parcel's starting temperature, then parcel temperature profile
        parcel_p, parcel_t, parcel_td, *parcel_i = parcel_func(pressure, temperature, dew_point_temperature)
        parcel_profile = mpcalc.parcel_profile(pressure, parcel_t, parcel_td)
    else:
        parcel_profile = None

    return parcel_profile

@check_units("[pressure]", "[length]", "[temperature]", "[temperature]")
def calc_lfc_height(
    pressure: PintList, height_msl: PintList, temperature: PintList, dew_point_temperature: PintList, parcel_func: Callable
) -> Tuple[int, pint.Quantity]:
    """
    Retrieve a parcel's level of free convection (lfc).

    Args:
        pressure:
            Total atmospheric pressure
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        temperature:
            Air temperature
        dew_point_temperature:
            Dew point temperature
        parcel_func:
            parcel profile retrieval callable via MetPy

    Returns:
        lfc:
            index of the last instance of negative buoyancy below the lfc
        lfc_z:
            height of the last instance of negative buoyancy below the lfc

    """

    # calculate the parcel's temperature profile
    parcel_profile = _get_parcel_profile(pressure, temperature, dew_point_temperature, parcel_func)

    # calculate the lfc, select the appropriate index & associated height
    lfc_p, lfc_t = mpcalc.lfc(pressure, temperature, dew_point_temperature, parcel_temperature_profile=parcel_profile)
    
    #press = np.round(np.asarray(P_sounding),8)
    nearestLFC=find_nearest(np.asarray(pressure),lfc_p.magnitude)
    lfc_idx = np.where(np.asarray(pressure)==nearestLFC)
    lfc_idx=lfc_idx[0][0]
    
    #lfc_idx = (pressure - lfc_p > 0).nonzero()[0][-1]
    lfc_z = height_msl[lfc_idx]

    return lfc_idx, lfc_z

@check_units("[pressure]", "[length]", "[temperature]", "[temperature]")
def calc_el_height(
    pressure: PintList, height_msl: PintList, temperature: PintList, dew_point_temperature: PintList, parcel_func: Callable
) -> Tuple[int, pint.Quantity]:
    """
    Retrieve a parcel's equilibrium level (el).

    Args:
        pressure:
            Total atmospheric pressure
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        temperature:
            Air temperature
        dew_point_temperature:
            Dew point temperature
        parcel_func:
            parcel profile retrieval callable via MetPy

    Returns:
        el_idx:
            index of the last instance of positive buoyancy below the el
        el_z:
            height of the last instance of positive buoyancy below the el

    """

    # calculate the parcel's temperature profile
    parcel_profile = _get_parcel_profile(pressure, temperature, dew_point_temperature, parcel_func)

    # calculate the el, select the appropriate index & associated height
    el_p, el_t = mpcalc.el(pressure, temperature, dew_point_temperature, parcel_temperature_profile=parcel_profile)
    
    #press = np.round(np.asarray(P_sounding),8)
    #nearestEL=(find_nearest(press,el_p.magnitude))
    #el_idx = np.where(press==nearestEL)
    #el_idx=el_idx[0][0]
    
    nearestEL=find_nearest(np.asarray(pressure),el_p.magnitude)
    el_idx = np.where(np.asarray(pressure)==nearestEL)
    el_idx=el_idx[0][0]
    
    #el_idx = (pressure - el_p > 0).nonzero()[0][-1]
    el_z = height_msl[el_idx]

    return el_idx, el_z

@check_units("[pressure]", "[speed]", "[speed]", "[length]")
def calc_sr_wind(pressure: PintList, u_wind: PintList, v_wind: PintList, height_msl: PintList) -> pint.Quantity:
    """
    Calculate the mean storm relative (as compared to Bunkers right motion) wind magnitude in the 0-1 km AGL layer

    Args:
        pressure:
            Total atmospheric pressure
        u_wind:
            X component of the wind
        v_wind
            Y component of the wind
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.

    Returns:
        sr_wind:
            0-1 km AGL average storm relative wind magnitude

    """
    height_agl = height_msl - height_msl[0]
    bunkers_right, _, _ = mpcalc.bunkers_storm_motion(pressure, u_wind, v_wind, height_agl)  # right, left, mean

    u_sr = u_wind - bunkers_right[0]  # u-component
    v_sr = v_wind - bunkers_right[1]  # v-component

    u_sr_1km = u_sr[np.nonzero((height_agl >= 0 * units("m")) & (height_agl <= 1000 * units("m")))]
    v_sr_1km = v_sr[np.nonzero((height_agl >= 0 * units("m")) & (height_agl <= 1000 * units("m")))]

    sr_wind = np.mean(mpcalc.wind_speed(u_sr_1km, v_sr_1km))

    return sr_wind

@check_units("[pressure]", "[length]", "[temperature]", "[mass]/[mass]")
def calc_mse(
    pressure: PintList, height_msl: PintList, temperature: PintList, specific_humidity: PintList
) -> Tuple[PintList, PintList]:
    """
    Calculate the moist static energy terms of interest.

    Args:
        pressure:
            Total atmospheric pressure
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        temperature:
            Air temperature
        specific_humidity:
            Specific humidity

    Returns:
        moist_static_energy_bar:
            Mean moist static energy from the surface to a layer
        moist_static_energy_star:
            Saturated moist static energy
    """

    # calculate MSE_bar
    moist_static_energy = mpcalc.moist_static_energy(height_msl, temperature, specific_humidity)
    moist_static_energy_bar = np.cumsum(moist_static_energy) / np.arange(1, len(moist_static_energy) + 1)
    moist_static_energy_bar = moist_static_energy_bar.to("J/kg")

    # calculate MSE*
    saturation_mixing_ratio = mpcalc.saturation_mixing_ratio(pressure, temperature)
    moist_static_energy_star = mpcalc.moist_static_energy(height_msl, temperature, saturation_mixing_ratio)
    moist_static_energy_star = moist_static_energy_star.to("J/kg")

    return moist_static_energy_bar, moist_static_energy_star

@check_units("[energy]/[mass]", "[energy]/[mass]", "[temperature]")
def calc_integral_arg(moist_static_energy_bar, moist_static_energy_star, temperature) -> PintList:
    """
    Calculate the contents of the integral defined in the NCAPE equation (54).

    Args:
        moist_static_energy_bar:
            Mean moist static energy from the surface to a layer
        moist_static_energy_star:
            Saturated moist static energy
        temperature:
            Air temperature

    Returns:
        integral_arg:
            Contents of integral defined in NCAPE eqn. 54

    """

    # NCAPE eqn 54 integrand, see compute_NCAPE.m L32
    integral_arg = -(earth_gravity / (dry_air_spec_heat_press * temperature)) * (
        moist_static_energy_bar - moist_static_energy_star
    )

    return integral_arg

@check_units("[length]/[time]**2", "[length]", "[dimensionless]", "[dimensionless]")
def calc_ncape(integral_arg: PintList, height_msl: PintList, lfc_idx: int, el_idx: int) -> pint.Quantity:
    """
    Calculate the buoyancy dilution potential (NCAPE)

    Args:
        integral_arg:
            Contents of integral defined in NCAPE eqn. 54
        height_msl:
            Atmospheric heights at the levels given by 'pressure'.
        lfc_idx:
            Index of the last instance of negative buoyancy below the lfc
        el_idx:
            Index of the last instance of positive buoyancy below the el

    Returns:
        ncape:
            Buoyancy dilution potential of the free troposphere (eqn. 54)
    """

    # see compute_NCAPE.m L41
    ncape = np.sum(
        (0.5 * integral_arg[lfc_idx:el_idx] + 0.5 * integral_arg[lfc_idx + 1 : el_idx + 1])
        * (height_msl[lfc_idx + 1 : el_idx + 1] - height_msl[lfc_idx:el_idx])
    )

    return ncape

@check_units("[speed]", "[dimensionless]", "[length]**2/[time]**2", "[energy]/[mass]")
def calc_ecape_a(sr_wind: PintList, psi: pint.Quantity, ncape: pint.Quantity, cape: pint.Quantity) -> pint.Quantity:
    """
    Calculate the entraining cape of a parcel

    Args:
        sr_wind:
            0-1 km AGL average storm relative wind magnitude
        psi:
            Parameter defined in eqn. 52, constant for a given equilibrium level
        ncape:
            Buoyancy dilution potential of the free troposphere (eqn. 54)
        cape:
            Convective available potential energy (CAPE, user-defined type)
    Returns:
        ecape:
            Entraining CAPE (eqn. 55)
    """

    # broken into terms for readability
    term_a = sr_wind**2 / 2.0
    term_b = (-1 - psi - (2 * psi / sr_wind**2) * ncape) / (4 * psi / sr_wind**2)
    term_c = (
        np.sqrt((1 + psi + (2 * psi / sr_wind**2) * ncape) ** 2 + 8 * (psi / sr_wind**2) * (cape - (psi * ncape)))
    ) / (4 * psi / sr_wind**2)

    ecape_a = term_a + term_b + term_c

    # set to 0 if negative
    return ecape_a.to("J/kg") if ecape_a >= 0 else 0

@check_units("[length]")
def calc_psi(el_z: pint.Quantity) -> pint.Quantity:
    """
    Calculate the constant psi as denoted in eqn. 52

    Args:
        el_z:
            height of the last instance of positive buoyancy below the el

    Returns:
        psi:
            Parameter defined in eqn. 52, constant for a given equilibrium level, see COMPUTE_ECAPE.m L88 (pitchfork)
    """

    # additional constants as denoted in section 4 step 1.
    sigma = 1.6 * units("dimensionless")
    alpha = 0.8 * units("dimensionless")
    l_mix = 120.0 * units("m")
    pr = (1.0 / 3.0) * units("dimensionless")  # prandtl number
    ksq = 0.18 * units("dimensionless")  # von karman constant

    psi = (ksq * alpha**2 * np.pi**2 * l_mix) / (4.0 * pr * sigma**2 * el_z)

    return psi

@check_units("[length]", "[pressure]", "[temperature]", "[mass]/[mass]", "[speed]", "[speed]")
def calc_ecape(
    height_msl: PintList,
    pressure: PintList,
    temperature: PintList,
    specific_humidity: PintList,
    u_wind: PintList,
    v_wind: PintList,
    cape_type: str = "most_unstable",
    undiluted_cape: pint.Quantity = None,
) -> pint.Quantity:
    """
    Calculate the entraining CAPE (ECAPE) of a parcel

    Parameters:
    ------------
        height_msl: np.ndarray[pint.Quantity]
            Atmospheric heights at the levels given by 'pressure' (MSL)
        pressure: np.ndarray[pint.Quantity]
            Total atmospheric pressure
        temperature: np.ndarray[pint.Quantity]
            Air temperature
        specific humidity: np.ndarray[pint.Quantity]
            Specific humidity
        u_wind: np.ndarray[pint.Quantity]
            X component of the wind
        v_wind np.ndarray[pint.Quantity]
            Y component of the wind
        cape_type: str
            Variation of CAPE desired. 'most_unstable' (default), 'surface_based', or 'mixed_layer'
        undiluted_cape: pint.Quantity
            User-provided undiluted CAPE value

    Returns:
    ----------
        ecape : 'pint.Quantity'
            Entraining CAPE
    """

    cape_func = {
        "most_unstable": mpcalc.most_unstable_cape_cin,
        "surface_based": mpcalc.surface_based_cape_cin,
        "mixed_layer": mpcalc.mixed_layer_cape_cin,
    }

    parcel_func = {
        "most_unstable": mpcalc.most_unstable_parcel,
        "surface_based": None,
        "mixed_layer": mpcalc.mixed_parcel,
    }

    # calculate cape
    dew_point_temperature = mpcalc.dewpoint_from_specific_humidity(pressure, temperature, specific_humidity)

    # whether the user has not / has overidden the cape calculations
    if not undiluted_cape:
        cape, _ = cape_func[cape_type](pressure, temperature, dew_point_temperature)
    else:
        cape = undiluted_cape

    # calculate the level of free convection (lfc) and equilibrium level (el) indexes
    lfc_idx, _ = calc_lfc_height(pressure, height_msl, temperature, dew_point_temperature, parcel_func[cape_type])
    el_idx, el_z = calc_el_height(pressure, height_msl, temperature, dew_point_temperature, parcel_func[cape_type])

    # calculate the buoyancy dilution potential (ncape)
    moist_static_energy_bar, moist_static_energy_star = calc_mse(pressure, height_msl, temperature, specific_humidity)
    integral_arg = calc_integral_arg(moist_static_energy_bar, moist_static_energy_star, temperature)
    ncape = calc_ncape(integral_arg, height_msl, lfc_idx, el_idx)

    # calculate the storm relative (sr) wind
    sr_wind = calc_sr_wind(pressure, u_wind, v_wind, height_msl)

    # calculate the entraining cape (ecape)
    psi = calc_psi(el_z)
    ecape_a = calc_ecape_a(sr_wind, psi, ncape, cape)

    return ecape_a

if __name__ == "__main__":
    pass

In [3]:
#UPDATED COMPOSITE PARAMETER FUNCTIONS
#These are updated sharppy functions, based on the equations for these parameters on the SPC mesoanalysis page

def scp(mucape, srh, ebwd, mucin):
    '''
    Supercell Composite Parameter (Thompson et al. 2003, 2007, 2012; Gropp and Davenport 2018)

    SCP_CIN = (muCAPE / 1000 J kg-1) * (ESRH / 50 m2 s-2) * (EBWD / 20 m s-1) * (-40 J kg-1 / muCIN)
    EBWD is divided by 20 m/s in the range of 10-20 m s-1. EBWD less than 10 m/s is set to zero, and EBWD greater than 20 m/s is set to one.
    The muCIN is set to 1.0 when muCIN is greater than -40 kg-1.

        Parameters
        ----------
        prof : profile object
            Profile object
        mucape : number, optional
            Most Unstable CAPE from the parcel class (J/kg) (optional)
        srh : number, optional
            the effective SRH from the winds.helicity function (m2/s2)
        ebwd : number, optional
            effective bulk wind difference (m/s)

        Returns
        -------
        scp : number
            supercell composite parameter   
    '''
    if ebwd > 20:
        ebwd_term = 1.0
    elif ebwd < 10:
        ebwd_term = 0.0
    else:
        ebwd_term = ebwd / 20.0
     
    if mucin > -40:
        muCIN_term = 1.0
    else:
        muCIN_term = -40.0 / mucin

    # Calculate SCP
    muCAPE_term = mucape / 1000.0
    esrh_term = srh / 50.0

    scp = muCAPE_term * esrh_term * ebwd_term * muCIN_term
    return scp

def stp_fixed(sbcape, sblcl, srh01, bwd6, sbcin):
    '''
    Significant Tornado Parameter (Thompson et al. 2012)
    STP = (sbCAPE/1500 J kg-1) * ((2000-sbLCL)/1000 m) * (SRH1/150 m2 s-2) * (6BWD/20 m s-1)* ((200+sbCIN)/150 J kg-1)

    The sbLCL term is set to 1.0 when sbLCL < 1000 m, and set to 0.0 when sbLCL > 2000 m
    the sbCIN term is set to 1.0 when sbCIN > -50 J kg-1, and set to 0.0 when sbCIN < -200
    the 6BWD term is capped at a value of 1.5 for 6BWD > 30 m/s, and set to 0.0 when 6BWD < 12.5 m/s. 
   
        Parameters
        ----------
        sbcape : number
            Surface based CAPE from the parcel class (J/kg)
        sblcl : number
            Surface based lifted condensation level (LCL) (m)
        srh01 : number
            Surface to 1 km storm relative helicity (m2/s2)
        bwd6 : number
            Bulk wind difference between 0 to 6 km (m/s)

        Returns
        -------
        stp_fixed : number
            signifcant tornado parameter (fixed-layer)
    '''
    
    # Calculate SBLCL term
    if sblcl < 1000.: # less than 1000. meters
        lcl_term = 1.0
    elif sblcl > 2000.: # greater than 2000. meters
        lcl_term = 0.0
    else:
        lcl_term = ((2000.-sblcl)/1000.)

    # Calculate 6BWD term
    if bwd6 > 30.0:  # Cap at 30 m/s
        bwd6_term = 1.5
    elif bwd6 < 12.5:  # Below 12.5 m/s
        bwd6_term = 0.0
    else:
        bwd6_term = bwd6 / 20.0

    cape_term = sbcape / 1500.
    srh_term = srh01 / 150.

    # Calculate SBCIN term
    if sbcin > -50.0:  # Less negative than -50 J/kg
        cin_term = 1.0
    elif sbcin < -200.0:  # More negative than -200 J/kg
        cin_term = 0.0
    else:
        cin_term = (200.0 + sbcin) / 150.0

    stp_fixed = cape_term * lcl_term * srh_term * bwd6_term * cin_term
   
    return stp_fixed
    
def stp_effective(mlcape, esrh, ebwd, mllcl, mlcinh, ebot_hght):
    '''
    STP effective layer (Thompson et al. 2012)
    STP = (mlCAPE/1500 J kg-1) * ((2000-mlLCL)/1000 m) * (ESRH/150 m2 s-2) * (EBWD/20 m s-1) * ((200+mlCIN)/150 J kg-1)
    The mlLCL term is set to 1.0 when mlLCL < 1000 m, and set to 0.0 when mlLCL > 2000 m
    the mlCIN term is set to 1.0 when mlCIN > -50 J kg-1, and set to 0.0 when mlCIN < -200; 
    the EBWD term is capped at a value of 1.5 for EBWD > 30 m s-1, and set to 0.0 when EBWD < 12.5 m s-1. 
    the entire index is set to 0.0 when the effective inflow base is above the ground.

        Parameters
        ----------
        mlcape : float
            Mixed-layer CAPE from the parcel class (J/kg)
        esrh : float
            effective storm relative helicity (m2/s2)
        ebwd : float
            effective bulk wind difference (m/s)
        mllcl : float
            mixed-layer lifted condensation level (m)
        mlcinh : float
            mixed-layer convective inhibition (J/kg)

        Returns
        -------
        stp_cin : number
            significant tornado parameter (unitless)

        See Also
        --------
        stp_fixed
    '''
    # If the effective inflow base is above ground, return 0.0 for STP
    if ebot_hght > 0.0:
        return 0.0

    cape_term = mlcape / 1500.
    eshr_term = esrh / 150.
    
    if ebwd < 12.5:
        ebwd_term = 0.
    elif ebwd > 30.:
        ebwd_term = 1.5
    else:
        ebwd_term  = ebwd / 20.

    if mllcl < 1000.:
        lcl_term = 1.0
    elif mllcl > 2000.:
        lcl_term = 0.0
    else:
        lcl_term = ((2000. - mllcl) / 1000.)

    if mlcinh > -50:
        cinh_term = 1.0
    elif mlcinh < -200:
        cinh_term = 0.0
    else:
        cinh_term = ((mlcinh + 200.0) / 150.0)

    stp_effective = np.maximum(cape_term * eshr_term * ebwd_term * lcl_term * cinh_term, 0.0)
    return stp_effective
 
def stp_500mSRH(mlcape, mllcl, srh_0_500m, ebwd, mlcinh, ebot_hght, etop_hght):
    """
    Calculate the STP using 0–500m SRH within the Effective Inflow Layer.
    Coffer et al. 2019: https://journals.ametsoc.org/view/journals/wefo/34/5/waf-d-19-0115_1.xml

    STP = (mlCAPE/1500 J kg-1) * ((2000-mlLCL)/1000 m) * (0-500 m SRH/75 m2 s-2) * (EBWD/20 m s-1) * ((200+mlCIN)/150 J kg-1)
    
    The 0-500 m SRH is limited to within the EIL, if it exists. 
    The mlLCL term is set to 1.0 when mlLCL < 1000 m, and set to 0.0 when mlLCL > 2000 m; 
    the mlCIN term is set to 1.0 when mlCIN > -50 J kg-1, and set to 0.0 when mlCIN < -200;
    the EBWD term is capped at a value of 1.5 for EBWD > 30 m s-1, and set to 0.0 when EBWD < 12.5 m s-1. 
    The entire index is set to 0.0 when the effective inflow base is above the ground.

    Parameters:
        mlcape (float): Mixed-layer CAPE (J/kg).
        mllcl (float): Mixed-layer LCL height (m).
        srh_0_500m (float): 0-500m storm-relative helicity (m²/s²).
        ebwd (float): Effective bulk wind difference (m/s).
        mlcinh (float): Mixed-layer CIN (J/kg).
        ebot_hght (float): Effective inflow layer base height (m).
        etop_hght (float): Effective inflow layer top height (m).

    Returns:
        float: STP value.
    """
    # If the effective inflow base is above ground, return 0.0 for STP
    if ebot_hght > 0.0:
        return 0.0

    # Limit 0-500m SRH within the effective inflow layer
    if etop_hght > 500.0:
        srh_term = srh_0_500m / 75.0
    else:
        srh_term = 0.0

    # Calculate mlLCL term
    if mllcl < 1000.0:
        lcl_term = 1.0
    elif mllcl > 2000.0:
        lcl_term = 0.0
    else:
        lcl_term = (2000.0 - mllcl) / 1000.0

    # Calculate mlCIN term
    if mlcinh > -50.0:
        cinh_term = 1.0
    elif mlcinh < -200.0:
        cinh_term = 0.0
    else:
        cinh_term = (mlcinh + 200.0) / 150.0

    # Calculate EBWD term
    if ebwd < 12.5:
        ebwd_term = 0.0
    elif ebwd > 30.0:
        ebwd_term = 1.5
    else:
        ebwd_term = ebwd / 20.0

    # Calculate other terms
    cape_term = mlcape / 1500.0

    # Final STP calculation
    stp = max(cape_term * lcl_term * srh_term * ebwd_term * cinh_term, 0.0)
    return stp

def vtp(mlcape, mllcl, esrh, ebwd, mlcinh, cape_0_3km, lapse_rate_0_3km, ebot_hght):
    """
    Calculate the Violent Tornado Parameter (VTP): https://www.spc.noaa.gov/publications/mosier/2018-JOM1.pdf.
    VTP = (mlCAPE/1500 J kg-1) * ((2000-mlLCL)/1000 m) * (ESRH/150 m2 s-2) * (EBWD/20 m s-1) * ((200+mlCIN)/150 J kg-1) * (0-3 km MLCAPE/50 J kg-1) * (0-3 km Lapse Rate/6.5 ℃ km-1)
    
    The 0-3 km lapse rate term is set to 2.0 when 0-3 km MLCAPE > 100 J kg-1.
    The mlLCL term is set to 1.0 when mlLCL < 1000 m, and set to 0 when mlLCL > 2000 m
    The mlCIN term is set to 1.0 when mlCIN > -50 J kg-1, and set to 0 when mlCIN < -200
    The EBWD term is capped at a value of 1.5 for EBWD > 30 m s-1, and set to 0 when EBWD < 12.5 m s-1.
    The entire index is set to 0 when the effective inflow base is above the ground.

    Parameters:
        mlcape (float): Mixed-layer CAPE (J/kg).
        mllcl (float): Mixed-layer LCL height (m).
        esrh (float): Effective-layer storm-relative helicity (m²/s²).
        ebwd (float): Effective bulk wind difference (m/s).
        mlcinh (float): Mixed-layer CIN (J/kg).
        cape_0_3km (float): 0–3 km mixed-layer CAPE (J/kg).
        lapse_rate_0_3km (float): 0–3 km lapse rate (°C/km).
        ebot_hght (float): Effective inflow layer base height (m).

    Returns:
        float: VTP value.
    """
    # If the effective inflow base is above ground, return 0.0 for VTP
    if ebot_hght > 0.0:
        return 0.0

    # Calculate mlLCL term
    if mllcl < 1000.0:
        lcl_term = 1.0
    elif mllcl > 2000.0:
        lcl_term = 0.0
    else:
        lcl_term = (2000.0 - mllcl) / 1000.0

    # Calculate mlCIN term
    if mlcinh > -50.0:
        cinh_term = 1.0
    elif mlcinh < -200.0:
        cinh_term = 0.0
    else:
        cinh_term = (mlcinh + 200.0) / 150.0

    # Calculate EBWD term
    if ebwd < 12.5:
        ebwd_term = 0.0
    elif ebwd > 30.0:
        ebwd_term = 1.5
    else:
        ebwd_term = ebwd / 20.0

    # Calculate 0–3 km lapse rate term
    if cape_0_3km > 100.0:
        lapse_rate_term = 2.0  # Lapse rate set to 2.0 if 0–3 km MLCAPE > 100 J/kg
    else:
        lapse_rate_term = lapse_rate_0_3km / 6.5

    # Calculate other terms
    cape_term = mlcape / 1500.0
    esrh_term = esrh / 150.0
    cape_0_3km_term = cape_0_3km / 50.0

    # Final VTP calculation
    vtp_value = max(
        cape_term
        * lcl_term
        * esrh_term
        * ebwd_term
        * cinh_term
        * cape_0_3km_term
        * lapse_rate_term,
        0.0,
    )
    return vtp_value

In [4]:
#Convergence Functions
def calculate_skewt_params(lev, hgt, T, Td, uwnd, vwnd, x, y, direction):
    """Calculate Skew-T parameters for a given grid direction."""
    if direction == "N":
        dx, dy = 1, 0
    elif direction == "S":
        dx, dy = -1, 0
    elif direction == "E":
        dx, dy = 0, 1
    elif direction == "W":
        dx, dy = 0, -1
    else:
        raise ValueError("Direction must be one of ['N', 'S', 'E', 'W']")

    p_skewt = lev[:, x + dx, y + dy] * units.hectopascal
    hgt_skewt = (hgt[:, x + dx, y + dy] - hgt[0, x + dx, y + dy]) * units.meter
    T_skewt = (T[:, x + dx, y + dy] * units.kelvin).to(units.degC)
    Td_skewt = Td[:, x + dx, y + dy].to(units.degC)
    Td_skewt[np.isnan(Td_skewt)] = 0 * units.degC
    u_skewt = (uwnd[:, x + dx, y + dy] * units.meter_per_second).to(units.knot)
    v_skewt = (vwnd[:, x + dx, y + dy] * units.meter_per_second).to(units.knot)
    #Omega = VVEL_up[:, x + dx, y + dy] * (units.pascal / units.second)
    #RH_skewt = relative_humidity_from_dewpoint(T_skewt, Td_skewt)

    #return p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt, Omega, RH_skewt
    return p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt

def calculate_parcel_profiles(p_skewt, hgt_skewt, T_skewt, Td_skewt, u_skewt, v_skewt):
    """
    Calculate SHARPpy parcel profiles and vertical profiles for MU, ML, and SB parcels.

    Returns:
        dict: Contains SHARPpy parcel data, vertical profiles, and LFC pressures.
    """
    # Ensure consistent lengths for all inputs
    min_len = min(len(p_skewt), len(hgt_skewt), len(T_skewt), len(Td_skewt), len(u_skewt), len(v_skewt))
    p_skewt = p_skewt[:min_len]
    hgt_skewt = hgt_skewt[:min_len]
    T_skewt = T_skewt[:min_len]
    Td_skewt = Td_skewt[:min_len]
    u_skewt = u_skewt[:min_len]
    v_skewt = v_skewt[:min_len]

    # Create SHARPpy profile
    prof = profile.create_profile(
        profile="default",
        pres=p_skewt,
        hght=hgt_skewt,
        tmpc=T_skewt,
        dwpc=Td_skewt,
        u=u_skewt,
        v=v_skewt,
        missing=-9999,
    )

    # Mixed-Layer (ML) parcel
    ml_p, ml_T, ml_Td = mixed_parcel(p_skewt, T_skewt, Td_skewt)
    ml_profile = parcel_profile(p_skewt, ml_T, ml_Td)
    mlpcl = params.parcelx(prof, flag=4)  # 100 mb Mean Layer Parcel
    mllfc_p = mlpcl.lfcpres*units('hectopascal')
    if isinstance(mllfc_p, np.ma.core.MaskedConstant):
        mllfc_p = 500*units('hectopascal')

    # Most-Unstable (MU) parcel
    mu_p, mu_T, mu_Td, _ = most_unstable_parcel(p_skewt, T_skewt, Td_skewt)
    mu_profile = parcel_profile(p_skewt, mu_T, mu_Td)
    mupcl = params.parcelx(prof, flag=3)  # Most-Unstable Parcel
    mulfc_p = mupcl.lfcpres*units('hectopascal')
    if isinstance(mulfc_p,np.ma.core.MaskedConstant):
        mulfc_p = 500*units('hectopascal')

    # Surface-Based (SB) parcel
    sbpcl = params.parcelx(prof, flag=1)  # Surface-Based Parcel
    sb_profile = parcel_profile(p_skewt, T_skewt[0], Td_skewt[0])
    sblfc_p = sbpcl.lfcpres*units('hectopascal')
    if isinstance(sblfc_p, np.ma.core.MaskedConstant):
        sblfc_p = mllfc_p
    
    # Convert vertical profiles to Celsius
    sb_profile = (sb_profile - 273.15 * units.kelvin).magnitude * units.degC
    ml_profile = (ml_profile - 273.15 * units.kelvin).magnitude * units.degC
    mu_profile = (mu_profile - 273.15 * units.kelvin).magnitude * units.degC

    # Return profiles, including SHARPpy parcel data and vertical profiles
    return {
        "sb": {"parcel": sbpcl, "profile": sb_profile, "lfc_p": sblfc_p, "base":p_skewt[0]},
        "ml": {"parcel": mlpcl, "profile": ml_profile, "lfc_p": mllfc_p, "base":ml_p},
        "mu": {"parcel": mupcl, "profile": mu_profile, "lfc_p": mulfc_p, "base":mu_p},
    }

# def calculate_mean_convergence(subcloudUV_layer, distEW, distNS, parcel_h, lfc_h):
#     """
#     Calculate mean convergence for given layers, distances, and vertical levels.

#     Parameters:
#         subcloudUV_layer (dict): Dictionary with subcloud UV layers for directions ('E', 'W', 'N', 'S').
#         distEW (float): East-West distance in meters.
#         distNS (float): North-South distance in meters.
#         parcel_h (float): Parcel height (h) in meters.
#         lfc_h (float): Level of free convection (h) in meters.

#     Returns:
#         float: Mean subcloud convergence.
#     """
#     # Calculate convergence at each vertical level
#     part1 = (subcloudUV_layer['E'][1] - subcloudUV_layer['W'][1]) / (distEW * units.meter)
#     part2 = (subcloudUV_layer['N'][2] - subcloudUV_layer['S'][2]) / (distNS * units.meter)
#     convergence = part1 + part2

#     # Calculate vertical distance and mean convergence
#     vertical_distance = lfc_h - parcel_h
#     mean_convergence = np.sum(convergence) / (vertical_distance * 1000)  # Convert to kilometers

#     return mean_convergence

def calculate_distances(lat, lon, x, y):
    """Calculate distances between adjacent grid points."""
    coordN = (lat[x + 1, y], lon[x + 1, y])
    coordS = (lat[x - 1, y], lon[x - 1, y])
    coordE = (lat[x, y + 1], lon[x, y + 1])
    coordW = (lat[x, y - 1], lon[x, y - 1])

    distNS = geodesic(coordN, coordS).m
    distEW = geodesic(coordE, coordW).m
    return distNS, distEW

# def calculate_surface_convergence(subcloudUV_layer, distEW, distNS):
#     """
#     Calculate parcel level (surface) convergence for mixed-layer or most-unstable parcels.
    
#     Parameters:
#         subcloudUV_layer (dict): Sub-cloud UV layers for all directions {'N', 'S', 'E', 'W'}.
#         distEW (float): Distance between eastern and western points (meters).
#         distNS (float): Distance between northern and southern points (meters).
        
#     Returns:
#         surface_convergence (Quantity): Surface convergence value.
#     """
#     # Calculate convergence components
#     part1 = (subcloudUV_layer['E'][1][0] - subcloudUV_layer['W'][1][0]) / (distEW * units('meter'))
#     part2 = (subcloudUV_layer['N'][2][0] - subcloudUV_layer['S'][2][0]) / (distNS * units('meter'))
    
#     # Total surface convergence
#     surface_convergence = part1 + part2
#     return surface_convergence

In [5]:
# #Elevated Stable Layer Calculation Functions

def compute_lapse_rates(prof, pres_array):
    lapse_rates = []
    for i in range(len(pres_array) - 1):
        lapse = sharppy.sharptab.params.lapse_rate(prof, lower=pres_array[i], upper=pres_array[i + 1])
        lapse_rates.append(lapse)
    return np.array(lapse_rates)

# Function to compute heights for a given pressure array
def compute_heights(prof, pres_array):
    heights = [sharppy.sharptab.interp.hght(prof, p) for p in pres_array]
    return np.array(heights)

# Function to calculate ESL depths
def compute_esl_depth(lapse_array, hght_array, threshold):
    esl_indices = np.where(lapse_array < threshold)[0]
    if len(esl_indices) > 0:
        esl_depth = hght_array[esl_indices[-1]] - hght_array[esl_indices[0]]
    else:
        esl_depth = 0
    return esl_depth

pres_array = np.array([900., 890., 880., 870., 860., 850., 840., 830., 820., 810., 800.,
                       790., 780., 770., 760., 750., 740., 730., 720., 710., 700., 690.,
                       680., 670., 660., 650., 640., 630., 620., 610., 600.])

In [6]:
##########################################################
# Pre Load HRRR Files to speed up parameter calculations #
##########################################################
def download_and_open_hrrr(year, month, day, hours, s3_path='s3://noaa-hrrr-bdp-pds', level_filter='hybrid'):
    """
    Download and open HRRR GRIB2 files for specified hours.
    
    Parameters:
        year (int): Year of the data
        month (int): Month of the data
        day (int): Day of the data
        hours (list): List of hours to download
        s3_path (str): Base S3 path to HRRR files
        level_filter (str): Filter for the level type (e.g., 'hybrid')
    
    Returns:
        dict: Dictionary of Xarray datasets
    """
    fs = s3fs.S3FileSystem(anon=True)
    datasets = {}
    
    for idx, hour in enumerate(hours):
        # Format the year, month, day, and hour
        year_str = f"{year}"
        month_str = f"{month:02}"
        day_str = f"{day:02}"
        hour_str = f"{hour:02}"
        
        # Construct the file path and local filename
        time_in = f"{year_str}{month_str}{day_str}"
        file_path = f"{s3_path}/hrrr.{time_in}/conus/hrrr.t{hour_str}z.wrfnatf00.grib2"
        local_fname = f"HRRR_{year_str}{month_str}{day_str}_{hour_str}UTC.grib2"
        
        # Download the file
        if not os.path.exists(local_fname):  # Avoid re-downloading
            fs.get(file_path, local_fname)
        
        # Open the file with Xarray
        datasets[f"c{idx}"] = xr.open_dataset(local_fname, filter_by_keys={'typeOfLevel': level_filter})
    
    return datasets

# Parameters
year = 2019
month = 5

# Download data for 22nd May
hours_22 = [18, 19, 20, 21, 22, 23]
d1 = download_and_open_hrrr(year, month, 22, hours_22)

# Download data for 23rd May
hours_23 = [0, 1, 2, 3, 4, 5, 6, 7, 8]
d2 = download_and_open_hrrr(year, month, 23, hours_23)

# # Download data for 22nd May
# hours_22 = [15, 18, 21, 22]
# d1 = download_and_open_hrrr(year, month, 22, hours_22)

# # Download data for 23rd May
# hours_23 = [0, 3]
# d2 = download_and_open_hrrr(year, month, 23, hours_23)

D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfradial1' loading failed:
cannot import name 'DataTree' from 'xarray' (D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\__init__.py)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'datamet' loading failed:
cannot import name 'DataTree' from 'xarray' (D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\__init__.py)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'furuno' loading failed:
cannot import name 'DataTree' from 'xarray' (D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\__init__.py)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
D:\Anaconda3\envs\THESIS\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWar

In [10]:
##########################################################
# Pre Load HRRR Files to speed up parameter calculations #
##########################################################
year=2019
month=5

fs = s3fs.S3FileSystem(anon=True)

times522 = [20,21,22,23]
d1={}
for m in range(0,len(times522)):
    day=22
    hour=times522[m]
    time_start = datetime(int(year), int(month), int(day), int(hour), 0) # Our specified time
    month = time_start.month
    if month < 10:
        month = '0'+str(month)
    if hour < 10:
        hour = '0'+str(hour)
        
    timeIN=str(str(year)+str(month)+str(day))
    files = np.array(fs.ls('s3://noaa-hrrr-bdp-pds/hrrr.'+str(timeIN)+'/conus/'))
    fileN = np.where(files=='noaa-hrrr-bdp-pds/hrrr.'+str(timeIN)+'/conus/hrrr.t'+str(hour)+'z.wrfnatf00.grib2')
    fs.get(files[fileN[0][0]], files[fileN[0][0]].split('/')[-1])
    fname=files[fileN[0][0]].split('/')[-1]
    d1["c{0}".format(m)] = xr.open_dataset(fname, filter_by_keys={'typeOfLevel': 'hybrid'})

times523 = [0,1,2,3]
d2={}
for n in range(0,len(times523)):
    day=23
    hour=times523[n]
    time_start = datetime(int(year), int(month), int(day), int(hour), 0) # Our specified time
    month = time_start.month
    if month < 10:
        month = '0'+str(month)
    if hour < 10:
        hour = '0'+str(hour)
        
    timeIN=str(str(year)+str(month)+str(day))
    files = np.array(fs.ls('s3://noaa-hrrr-bdp-pds/hrrr.'+str(timeIN)+'/conus/'))
    fileN = np.where(files=='noaa-hrrr-bdp-pds/hrrr.'+str(timeIN)+'/conus/hrrr.t'+str(hour)+'z.wrfnatf00.grib2')
    fs.get(files[fileN[0][0]], files[fileN[0][0]].split('/')[-1])
    fname=files[fileN[0][0]].split('/')[-1]
    d2["c{0}".format(n)] = xr.open_dataset(fname, filter_by_keys={'typeOfLevel': 'hybrid'})
    
#print(d1['c0'].attrs)

In [27]:
hour = 21

if hour==20:
    c=d1['c0']
elif hour==21:
    c=d1['c1']
elif hour==22:
    c=d1['c2']
elif hour==23:
    c=d1['c3']
elif hour==0:
    c=d2['c0'] 
elif hour==1:
    c=d2['c1'] 
elif hour==2:
    c=d2['c2'] 
elif hour==3:
    c=d2['c3']            
else:
    print("WRONG HOUR")

time_start = datetime(int(year), int(month), int(day), int(hour), 0) # Our specified time
hour = time_start.hour
if hour < 10:
    hour = '0'+str(hour)
day = time_start.day
if day < 10:
    day = '0'+str(day)
month = time_start.month
if month < 10:
    month = '0'+str(month)

timeIN=str(str(year)+str(month)+str(day))

##########################################################
# eCAPE = []
# u_700 = []
# v_700 = []
# u_SFC = []
# v_SFC = []
# esl_25 = []
# RH_2500m_5km = []
# bwd_0_1 = []
# srh_0_1 = []
stp_fl = []
stp_500 = []
#########################################################   
lev=np.asarray(c.variables['hybrid'][:]) # 0 = lowest level, 49 = highest level
lat=np.asarray(c.variables['latitude'])
lon=((np.asarray(c.variables['longitude'])*-1)+360)*-1

T=np.asarray(c.variables['t'][:]) #K #temperature
q=np.asarray(c.variables['q'][:]) # kg kg**-1 #specific humidity
uwnd=np.asarray(c.variables['u'][:]) #m/s #u wind
vwnd=np.asarray(c.variables['v'][:]) #m/s #v wind
hgt=np.asarray(c.variables['gh'][:]) #geopotential meters #geopotential height
#VVEL_up=(np.asarray(c.variables['w'][:])) #Pa s**-1 #vertical velocity  
lev=(np.asarray(c.variables['pres'][:])/100.) #hPa #pressure
Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)

NElat, NElon = 33.0, -93.0
NWlat, NWlon = 33.0, -101.0
SElat, SElon = 38.0, -93.0
SWlat, SWlon = 38.0, -101.0

dlonNE = np.abs(lon - NElon)
dlatNE = np.abs(lat - NElat)
cordmaxNE = np.maximum(dlonNE,dlatNE)
xNE,yNE=np.where(cordmaxNE == np.min(cordmaxNE))
xNE=xNE[0]
yNE=yNE[0]

dlonNW = np.abs(lon - NWlon)
dlatNW = np.abs(lat - NWlat)
cordmaxNW = np.maximum(dlonNW,dlatNW)
xNW,yNW=np.where(cordmaxNW == np.min(cordmaxNW))
xNW=xNW[0]
yNW=yNW[0]

dlonSE = np.abs(lon - SElon)
dlatSE = np.abs(lat - SElat)
cordmaxSE = np.maximum(dlonSE,dlatSE)
xSE,ySE=np.where(cordmaxSE == np.min(cordmaxSE))
xSE=xSE[0]
ySE=ySE[0]

dlonSW = np.abs(lon - SWlon)
dlatSW = np.abs(lat - SWlat)
cordmaxSW = np.maximum(dlonSW,dlatSW)
xSW,ySW=np.where(cordmaxSW == np.min(cordmaxSW))
xSW=xSW[0]
ySW=ySW[0]

xNE = 327
xSE = 513
ySE = 1040
ySW = 790

for i in range (327,513):
    for j in range (790,1040):
        P_sounding = lev[:,i,j]* units.hectopascal
        T_sounding = (T[:,i,j]* units.kelvin).to(units.degC)
        Td_sounding = Td[:,i,j].to(units.degC)
        Td_sounding[np.isnan(Td_sounding)] = 0*units('degC')
        hgt_sounding = (hgt[:,i,j] - hgt[0,i,j])* units.meter
        #msl_sounding = (hgt[:,i,j])* units.meter
        u_sounding = (uwnd[:,i,j]* units.meter_per_second).to(units.knot)
        v_sounding = (vwnd[:,i,j]* units.meter_per_second).to(units.knot)
        #Omega=VVEL_up[:,x,y]*(units.pascal/units.second)
        #RH_skewt = relative_humidity_from_dewpoint(T_sounding, Td_sounding)
        #Q_skewt = mixing_ratio_from_relative_humidity(P_sounding, T_sounding, RH_skewt)

        prof = profile.create_profile(profile='default', pres=P_sounding, hght=hgt_sounding, tmpc=T_sounding, dwpc=Td_sounding, u=u_sounding, v=v_sounding, missing=-9999)
        #mupcl = params.parcelx(prof, flag=3) # Most-Unstable Parcel
        mlpcl = params.parcelx(prof, flag=4)
        sbpcl = params.parcelx(prof, flag=1)
#         wind700 = interp.components(prof, 700.)
#         windSFC = interp.components(prof, prof.pres[prof.sfc])
#         u700 = wind700[0]
#         v700 = wind700[1]
#         uSFC = windSFC[0]
#         vSFC = windSFC[1]

        #EIL Stuff
        EILb, EILt = sharppy.sharptab.params.effective_inflow_layer(prof)
        ebot_hght = interp.to_agl(prof, interp.hght(prof, EILb))
        etop_hght = interp.to_agl(prof, interp.hght(prof, EILt))

        #METPY PARCEL INFORMATION
        #Interpolate new vertical profiles with the mlpcl and mupcl pressures, temps, and dewpoints as the starting values for each
        #ml_p, ml_T, ml_Td = mixed_parcel(P_sounding, T_sounding, Td_sounding) #Pres, Temp, and Td of a 100 hPa depth mixed parcels
        #ml_profile = parcel_profile(P_sounding, ml_T, ml_Td).to(units.degC)
        #ml_parcel_h = pressure_to_height_std(ml_p)
        #ml_parcel_h = ml_parcel_h.to(units.meter)

        #mu_p, mu_T, mu_Td, mu_index = most_unstable_parcel(P_sounding, T_sounding, Td_sounding)
        #mu_profile = parcel_profile(P_sounding, mu_T, mu_Td).to(units.degC)
        #mu_parcel_h = pressure_to_height_std(mu_p)
        #mu_parcel_h = mu_parcel_h.to(units.meter)

        #sb_profile = parcel_profile(P_sounding, T_sounding[0], Td_sounding[0])

        sfc = prof.pres[prof.sfc]
        p500m = interp.pres(prof, interp.to_msl(prof, 500.))
        p1km = interp.pres(prof, interp.to_msl(prof, 1000.))
        p6km = interp.pres(prof, interp.to_msl(prof, 6000.))

        #################################################################################################################
        #LAPSE RATES
        #################################################################################################################
        # Compute ESL depths
#         presArray=np.array([890.,880.,870.,860.,850.,840.,830.,820.,810.,800.,790.,780.,770.,760.,750.,740.,730.,720.,710.,700.,
#                        690.,680.,670.,660.,650.,640.,630.,620.,610.,600.])

#         lapse1 = sharppy.sharptab.params.lapse_rate(prof, lower = 900., upper =890.)
#         lapse2 = sharppy.sharptab.params.lapse_rate(prof, lower = 890., upper =880.)
#         lapse3 = sharppy.sharptab.params.lapse_rate(prof, lower = 880., upper =870.)
#         lapse4 = sharppy.sharptab.params.lapse_rate(prof, lower = 870., upper =860.)
#         lapse5 = sharppy.sharptab.params.lapse_rate(prof, lower = 860., upper =850.)
#         lapse6 = sharppy.sharptab.params.lapse_rate(prof, lower = 850., upper =840.)
#         lapse7 = sharppy.sharptab.params.lapse_rate(prof, lower = 840., upper =830.)
#         lapse8 = sharppy.sharptab.params.lapse_rate(prof, lower = 830., upper =820.)
#         lapse9 = sharppy.sharptab.params.lapse_rate(prof, lower = 820., upper =810.)
#         lapse10 = sharppy.sharptab.params.lapse_rate(prof, lower = 810., upper =800.)
#         lapse11 = sharppy.sharptab.params.lapse_rate(prof, lower = 800., upper =790.)
#         lapse12 = sharppy.sharptab.params.lapse_rate(prof, lower = 790., upper =780.)
#         lapse13 = sharppy.sharptab.params.lapse_rate(prof, lower = 780., upper =770.)
#         lapse14 = sharppy.sharptab.params.lapse_rate(prof, lower = 770., upper =760.)
#         lapse15 = sharppy.sharptab.params.lapse_rate(prof, lower = 760., upper =750.)
#         lapse16 = sharppy.sharptab.params.lapse_rate(prof, lower = 750., upper =740.)
#         lapse17 = sharppy.sharptab.params.lapse_rate(prof, lower = 740., upper =730.)
#         lapse18 = sharppy.sharptab.params.lapse_rate(prof, lower = 730., upper =720.)
#         lapse19 = sharppy.sharptab.params.lapse_rate(prof, lower = 720., upper =710.)
#         lapse20 = sharppy.sharptab.params.lapse_rate(prof, lower = 710., upper =700.)
#         lapse21 = sharppy.sharptab.params.lapse_rate(prof, lower = 700., upper =690.)
#         lapse22 = sharppy.sharptab.params.lapse_rate(prof, lower = 690., upper =680.)
#         lapse23 = sharppy.sharptab.params.lapse_rate(prof, lower = 680., upper =670.)
#         lapse24 = sharppy.sharptab.params.lapse_rate(prof, lower = 670., upper =660.)
#         lapse25 = sharppy.sharptab.params.lapse_rate(prof, lower = 660., upper =650.)
#         lapse26 = sharppy.sharptab.params.lapse_rate(prof, lower = 650., upper =640.)
#         lapse27 = sharppy.sharptab.params.lapse_rate(prof, lower = 640., upper =630.)
#         lapse28 = sharppy.sharptab.params.lapse_rate(prof, lower = 630., upper =620.)
#         lapse29 = sharppy.sharptab.params.lapse_rate(prof, lower = 620., upper =610.)
#         lapse30 = sharppy.sharptab.params.lapse_rate(prof, lower = 610., upper =600.)

#         hght900 = interp.hght(prof, 900.)
#         hght890 = interp.hght(prof, 890.)
#         hght880 = interp.hght(prof, 880.)
#         hght870 = interp.hght(prof, 870.)
#         hght860 = interp.hght(prof, 860.)
#         hght850 = interp.hght(prof, 850.)
#         hght840 = interp.hght(prof, 840.)
#         hght830 = interp.hght(prof, 830.)
#         hght820 = interp.hght(prof, 820.)
#         hght810 = interp.hght(prof, 810.)
#         hght800 = interp.hght(prof, 800.)
#         hght790 = interp.hght(prof, 790.)
#         hght780 = interp.hght(prof, 780.)
#         hght770 = interp.hght(prof, 770.)
#         hght760 = interp.hght(prof, 760.)
#         hght750 = interp.hght(prof, 750.)
#         hght740 = interp.hght(prof, 740.)
#         hght730 = interp.hght(prof, 730.)
#         hght720 = interp.hght(prof, 720.)
#         hght710 = interp.hght(prof, 710.)
#         hght700 = interp.hght(prof, 700.)
#         hght690 = interp.hght(prof, 690.)
#         hght680 = interp.hght(prof, 680.)
#         hght670 = interp.hght(prof, 670.)
#         hght660 = interp.hght(prof, 660.)
#         hght650 = interp.hght(prof, 650.)
#         hght640 = interp.hght(prof, 640.)
#         hght630 = interp.hght(prof, 630.)
#         hght620 = interp.hght(prof, 620.)
#         hght610 = interp.hght(prof, 610.)
#         hght600 = interp.hght(prof, 600.)

#         lapseArray=np.array([lapse1,lapse2,lapse3,lapse4,lapse5,lapse6,lapse7,lapse8,lapse9,lapse10,
#                             lapse11,lapse12,lapse13,lapse14,lapse15,lapse16,lapse17,lapse18,lapse19,lapse20,
#                             lapse21,lapse22,lapse23,lapse24,lapse25,lapse26,lapse27,lapse28,lapse29,lapse30])

#         hghtArray=np.array([hght890,hght880,hght870,hght860,hght850,hght840,hght830,hght820,hght810,hght800,
#                            hght790,hght780,hght770,hght760,hght750,hght740,hght730,hght720,hght710,hght700,
#                            hght690,hght680,hght670,hght660,hght650,hght640,hght630,hght620,hght610,hght600,])

#         ESL25 = np.asarray(np.where(lapseArray < 2.5))
#         if len(ESL25[0]) != 0:
#             ESL25hgtArray = hghtArray[ESL25[0][0]:ESL25[0][-1]]
#             esl_25_depth = hghtArray[ESL25[0][-1]] - hghtArray[ESL25[0][0]]
#         else:
#             esl_25_depth = 0

        #ESL40 = np.asarray(np.where(lapseArray < 4.0))
        #if len(ESL40[0]) != 0:
        #    ESL40hgtArray = hghtArray[ESL40[0][0]:ESL40[0][-1]]
        #    esl_40_depth = hghtArray[ESL40[0][-1]] - hghtArray[ESL40[0][0]]
        #else:
        #    esl_40_depth = 0
        
        #p2500m = interp.pres(prof, interp.to_msl(prof, 2500.))
        #p5km = interp.pres(prof, interp.to_msl(prof, 5000.))
        #mean2500m_5km_RH = sharppy.sharptab.params.mean_relh(prof, pbot=p2500m, ptop=p5km)
        #################################################################################################################
        #################################################################################################################
        #mlCAPECIN_1500m_5km = sharppy.sharptab.params.cape(prof, pbot=p1500m, ptop=p5km, flag=4)   
        #################################################################################################################
        # BUOYANCY
        #################################################################################################################
        #sbcape = sbpcl.bplus
        #sb3kmCAPE = sbpcl.b3km

        #ecape = calc_ecape(msl_sounding, P_sounding, T_sounding, Q_skewt, u_sounding, v_sounding)
        #mlCAPE_1500m_5km = mlCAPECIN_1500m_5km.bplus

        #################################################################################################################
        #INHIBITION
        #################################################################################################################
        #mlcin = mlpcl.bminus
        #mucin =  mupcl.bminus
        #sbcin = sbpcl.bminus
        #################################################################################################################
        # MOTION
        #################################################################################################################
        #calculate storm motion using Bunkers et. al 2014
        #using effective inflow base as base, and pressure weighted mean wind    
        srwind = params.bunkers_storm_motion(prof)
        #this gives us u and v motion vectors for right movers [0 and 1] and left movers [2 and 3]

        #bunkersU = srwind[0]
        #bunkersV = srwind[1]
        #vectorMag = math.sqrt(((bunkersU)**2) + ((bunkersV)**2))
        #angle = math.degrees(math.atan2(bunkersV,bunkersU)) #angle from X axis
        #################################################################################################################
        #SRH Calculations
        #################################################################################################################
        srh500m = winds.helicity(prof, 0, 500., stu = srwind[0], stv = srwind[1])[0]
        srh1km = winds.helicity(prof, 0, 1000., stu = srwind[0], stv = srwind[1])[0]
        #srh3km = winds.helicity(prof, 0, 3000., stu = srwind[0], stv = srwind[1])[0]
        effective_srh = winds.helicity(prof, ebot_hght, etop_hght, stu = srwind[0], stv = srwind[1])[0]
        #################################################################################################################
        #BULK SHEAR
        #################################################################################################################
        #Get shear magnitudes
        ml_halfELpres = interp.pres(prof, interp.to_msl(prof, mlpcl.elhght/2.))
        ml_ebwd = winds.wind_shear(prof, pbot=EILb, ptop=ml_halfELpres)
        ml_ebwspd = utils.mag(ml_ebwd[0], ml_ebwd[1])

        sfc_1km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p1km)
        sfc1shear = utils.mag(sfc_1km_shear[0], sfc_1km_shear[1]) 
        sfc_6km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p6km)
        sfc6shear = utils.mag(sfc_6km_shear[0], sfc_6km_shear[1])
 
        #################################################################################################################
        # COMPOSITES    
        #################################################################################################################
        #SCP = scp(mupcl.bplus, effective_srh, mu_ebwspd, mupcl.bminus)
        STP_FL = stp_fixed(sbpcl.bplus, sbpcl.lclhght, srh1km, sfc6shear, sbpcl.bminus)
        #STP_EL = stp_effective(mlpcl.bplus, effective_srh, ml_ebwspd, mlpcl.lclhght, mlpcl.bminus, ebot_hght)

        if etop_hght > 500.0:
            srh500mEIL = srh500m  # Entire 0–500m layer is within the EIL
        else:
            srh500mEIL = effective_srh  # Only use the SRH within the EIL

        STP_500 = stp_500mSRH(mlpcl.bplus, mlpcl.lclhght, srh500mEIL, ml_ebwspd, mlpcl.bminus, ebot_hght, etop_hght)
        #VTP = vtp(mlpcl.bplus, mlpcl.lclhght, effective_srh, ml_ebwspd, mlpcl.bminus, mlpcl.b3km, lapse0_3km, ebot_hght)

        #eCAPE.append(ecape)
        #u_700.append(u700) #KNOTS
        #v_700.append(v700) #KNOTS
        #u_SFC.append(uSFC) #KNOTS
        #v_SFC.append(vSFC) #KNOTS       
        #esl_25.append(esl_25_depth)
        #RH_2500m_5km.append(mean2500m_5km_RH)
        #bwd_0_1.append(sfc1shear)
        #srh_0_1.append(srh1km)
        stp_fl.append(STP_FL)
        stp_500.append(STP_500)
        
        print(i,j)

        j+=1
    i+=1
    
#eCAPE = np.asarray(eCAPE)
# u_700 = np.asarray(u_700)
# v_700 = np.asarray(v_700)
# u_SFC = np.asarray(u_SFC)
# v_SFC = np.asarray(v_SFC)
# esl_25 = np.asarray(esl_25)
# RH_2500m_5km = np.asarray(RH_2500m_5km)
# bwd_0_1 = np.asarray(bwd_0_1)
# srh_0_1 = np.asarray(srh_0_1)
stp_fl = np.asarray(stp_fl)
stp_500 = np.asarray(stp_500)

#eCAPE = np.array_split(eCAPE, 186)
# u_700 = np.array_split(u_700, 186)
# v_700 = np.array_split(v_700, 186)
# u_SFC = np.array_split(u_SFC, 186)
# v_SFC = np.array_split(v_SFC, 186)
# esl_25 = np.array_split(esl_25, 186)
# RH_2500m_5km = np.array_split(RH_2500m_5km, 186)
# bwd_0_1 = np.array_split(bwd_0_1, 186)
# srh_0_1 = np.array_split(srh_0_1, 186)
stp_fl = np.array_split(stp_fl, 186)
stp_500 = np.array_split(stp_500, 186)

#hr20_eCAPE = np.asarray(eCAPE)
# hr20_u_700 = np.asarray(u_700)
# hr20_v_700 = np.asarray(v_700)
# hr20_u_SFC = np.asarray(u_SFC)
# hr20_v_SFC = np.asarray(v_SFC)
# hr20_esl_25 = np.asarray(esl_25)
# hr20_RH_2500m_5km = np.asarray(RH_2500m_5km)
# hr20_bwd_0_1 = np.asarray(bwd_0_1)
# hr20_srh_0_1 = np.asarray(srh_0_1)
hr20_stp_fl = np.asarray(stp_fl)
hr20_stp_500 = np.asarray(stp_500)

#np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_20\hr20_eCAPE.csv",hr20_eCAPE,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_u_700.csv",hr20_u_700,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_v_700.csv",hr20_v_700,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_u_SFC.csv",hr20_u_SFC,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_v_SFC.csv",hr20_v_SFC,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_esl_25.csv",hr20_esl_25,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_RH_2500m_5km.csv",hr20_RH_2500m_5km,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_bwd_0_1.csv",hr20_bwd_0_1,delimiter=",",fmt='%s')
# np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr21_srh_0_1.csv",hr20_srh_0_1,delimiter=",",fmt='%s')
np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr20_stp_fl.csv",hr20_stp_fl,delimiter=",",fmt='%s')
np.savetxt(r"D:\THESIS\__UNL MS Thesis (MAIN)__\5 - Spatial Parameter Analysis\NEW_ARRAY\Hourly_Array_21\hr20_stp_500.csv",hr20_stp_500,delimiter=",",fmt='%s')

C:\Users\kyled\AppData\Local\Temp\ipykernel_10920\4137390217.py:59: UserWarning: Relative humidity >120%, ensure proper units.
  Td=dewpoint_from_specific_humidity(lev* units('hPa'),T* units('kelvin'),q)


327 790
327 791
327 792
327 793
327 794
327 795
327 796
327 797
327 798
327 799
327 800
327 801
327 802
327 803
327 804
327 805
327 806
327 807
327 808
327 809
327 810
327 811
327 812
327 813
327 814
327 815
327 816
327 817
327 818
327 819
327 820
327 821
327 822
327 823
327 824
327 825
327 826
327 827
327 828
327 829
327 830
327 831
327 832
327 833
327 834
327 835
327 836
327 837
327 838
327 839
327 840
327 841
327 842
327 843
327 844
327 845
327 846
327 847
327 848
327 849
327 850
327 851
327 852
327 853
327 854
327 855
327 856
327 857
327 858
327 859
327 860
327 861
327 862
327 863
327 864
327 865
327 866
327 867
327 868
327 869
327 870
327 871
327 872
327 873
327 874
327 875
327 876
327 877
327 878
327 879
327 880
327 881
327 882
327 883
327 884
327 885
327 886
327 887
327 888
327 889
327 890
327 891
327 892
327 893
327 894
327 895
327 896
327 897
327 898
327 899
327 900
327 901
327 902
327 903
327 904
327 905
327 906
327 907
327 908
327 909
327 910
327 911
327 912
327 913
327 914


331 796
331 797
331 798
331 799
331 800
331 801
331 802
331 803
331 804
331 805
331 806
331 807
331 808
331 809
331 810
331 811
331 812
331 813
331 814
331 815
331 816
331 817
331 818
331 819
331 820
331 821
331 822
331 823
331 824
331 825
331 826
331 827
331 828
331 829
331 830
331 831
331 832
331 833
331 834
331 835
331 836
331 837
331 838
331 839
331 840
331 841
331 842
331 843
331 844
331 845
331 846
331 847
331 848
331 849
331 850
331 851
331 852
331 853
331 854
331 855
331 856
331 857
331 858
331 859
331 860
331 861
331 862
331 863
331 864
331 865
331 866
331 867
331 868
331 869
331 870
331 871
331 872
331 873
331 874
331 875
331 876
331 877
331 878
331 879
331 880
331 881
331 882
331 883
331 884
331 885
331 886
331 887
331 888
331 889
331 890
331 891
331 892
331 893
331 894
331 895
331 896
331 897
331 898
331 899
331 900
331 901
331 902
331 903
331 904
331 905
331 906
331 907
331 908
331 909
331 910
331 911
331 912
331 913
331 914
331 915
331 916
331 917
331 918
331 919
331 920


335 801
335 802
335 803
335 804
335 805
335 806
335 807
335 808
335 809
335 810
335 811
335 812
335 813
335 814
335 815
335 816
335 817
335 818
335 819
335 820
335 821
335 822
335 823
335 824
335 825
335 826
335 827
335 828
335 829
335 830
335 831
335 832
335 833
335 834
335 835
335 836
335 837
335 838
335 839
335 840
335 841
335 842
335 843
335 844
335 845
335 846
335 847
335 848
335 849
335 850
335 851
335 852
335 853
335 854
335 855
335 856
335 857
335 858
335 859
335 860
335 861
335 862
335 863
335 864
335 865
335 866
335 867
335 868
335 869
335 870
335 871
335 872
335 873
335 874
335 875
335 876
335 877
335 878
335 879
335 880
335 881
335 882
335 883
335 884
335 885
335 886
335 887
335 888
335 889
335 890
335 891
335 892
335 893
335 894
335 895
335 896
335 897
335 898
335 899
335 900
335 901
335 902
335 903
335 904
335 905
335 906
335 907
335 908
335 909
335 910
335 911
335 912
335 913
335 914
335 915
335 916
335 917
335 918
335 919
335 920
335 921
335 922
335 923
335 924
335 925


339 806
339 807
339 808
339 809
339 810
339 811
339 812
339 813
339 814
339 815
339 816
339 817
339 818
339 819
339 820
339 821
339 822
339 823
339 824
339 825
339 826
339 827
339 828
339 829
339 830
339 831
339 832
339 833
339 834
339 835
339 836
339 837
339 838
339 839
339 840
339 841
339 842
339 843
339 844
339 845
339 846
339 847
339 848
339 849
339 850
339 851
339 852
339 853
339 854
339 855
339 856
339 857
339 858
339 859
339 860
339 861
339 862
339 863
339 864
339 865
339 866
339 867
339 868
339 869
339 870
339 871
339 872
339 873
339 874
339 875
339 876
339 877
339 878
339 879
339 880
339 881
339 882
339 883
339 884
339 885
339 886
339 887
339 888
339 889
339 890
339 891
339 892
339 893
339 894
339 895
339 896
339 897
339 898
339 899
339 900
339 901
339 902
339 903
339 904
339 905
339 906
339 907
339 908
339 909
339 910
339 911
339 912
339 913
339 914
339 915
339 916
339 917
339 918
339 919
339 920
339 921
339 922
339 923
339 924
339 925
339 926
339 927
339 928
339 929
339 930


...

499 1012
499 1013
499 1014
499 1015
499 1016
499 1017
499 1018
499 1019
499 1020
499 1021
499 1022
499 1023
499 1024
499 1025
499 1026
499 1027
499 1028
499 1029
499 1030
499 1031
499 1032
499 1033
499 1034
499 1035
499 1036
499 1037
499 1038
499 1039
500 790
500 791
500 792
500 793
500 794
500 795
500 796
500 797
500 798
500 799
500 800
500 801
500 802
500 803
500 804
500 805
500 806
500 807
500 808
500 809
500 810
500 811
500 812
500 813
500 814
500 815
500 816
500 817
500 818
500 819
500 820
500 821
500 822
500 823
500 824
500 825
500 826
500 827
500 828
500 829
500 830
500 831
500 832
500 833
500 834
500 835
500 836
500 837
500 838
500 839
500 840
500 841
500 842
500 843
500 844
500 845
500 846
500 847
500 848
500 849
500 850
500 851
500 852
500 853
500 854
500 855
500 856
500 857
500 858
500 859
500 860
500 861
500 862
500 863
500 864
500 865
500 866
500 867
500 868
500 869
500 870
500 871
500 872
500 873
500 874
500 875
500 876
500 877
500 878
500 879
500 880
500 881
500 882
500 

503 1016
503 1017
503 1018
503 1019
503 1020
503 1021
503 1022
503 1023
503 1024
503 1025
503 1026
503 1027
503 1028
503 1029
503 1030
503 1031
503 1032
503 1033
503 1034
503 1035
503 1036
503 1037
503 1038
503 1039
504 790
504 791
504 792
504 793
504 794
504 795
504 796
504 797
504 798
504 799
504 800
504 801
504 802
504 803
504 804
504 805
504 806
504 807
504 808
504 809
504 810
504 811
504 812
504 813
504 814
504 815
504 816
504 817
504 818
504 819
504 820
504 821
504 822
504 823
504 824
504 825
504 826
504 827
504 828
504 829
504 830
504 831
504 832
504 833
504 834
504 835
504 836
504 837
504 838
504 839
504 840
504 841
504 842
504 843
504 844
504 845
504 846
504 847
504 848
504 849
504 850
504 851
504 852
504 853
504 854
504 855
504 856
504 857
504 858
504 859
504 860
504 861
504 862
504 863
504 864
504 865
504 866
504 867
504 868
504 869
504 870
504 871
504 872
504 873
504 874
504 875
504 876
504 877
504 878
504 879
504 880
504 881
504 882
504 883
504 884
504 885
504 886
504 887


507 1020
507 1021
507 1022
507 1023
507 1024
507 1025
507 1026
507 1027
507 1028
507 1029
507 1030
507 1031
507 1032
507 1033
507 1034
507 1035
507 1036
507 1037
507 1038
507 1039
508 790
508 791
508 792
508 793
508 794
508 795
508 796
508 797
508 798
508 799
508 800
508 801
508 802
508 803
508 804
508 805
508 806
508 807
508 808
508 809
508 810
508 811
508 812
508 813
508 814
508 815
508 816
508 817
508 818
508 819
508 820
508 821
508 822
508 823
508 824
508 825
508 826
508 827
508 828
508 829
508 830
508 831
508 832
508 833
508 834
508 835
508 836
508 837
508 838
508 839
508 840
508 841
508 842
508 843
508 844
508 845
508 846
508 847
508 848
508 849
508 850
508 851
508 852
508 853
508 854
508 855
508 856
508 857
508 858
508 859
508 860
508 861
508 862
508 863
508 864
508 865
508 866
508 867
508 868
508 869
508 870
508 871
508 872
508 873
508 874
508 875
508 876
508 877
508 878
508 879
508 880
508 881
508 882
508 883
508 884
508 885
508 886
508 887
508 888
508 889
508 890
508 891
508 

511 1025
511 1026
511 1027
511 1028
511 1029
511 1030
511 1031
511 1032
511 1033
511 1034
511 1035
511 1036
511 1037
511 1038
511 1039
512 790
512 791
512 792
512 793
512 794
512 795
512 796
512 797
512 798
512 799
512 800
512 801
512 802
512 803
512 804
512 805
512 806
512 807
512 808
512 809
512 810
512 811
512 812
512 813
512 814
512 815
512 816
512 817
512 818
512 819
512 820
512 821
512 822
512 823
512 824
512 825
512 826
512 827
512 828
512 829
512 830
512 831
512 832
512 833
512 834
512 835
512 836
512 837
512 838
512 839
512 840
512 841
512 842
512 843
512 844
512 845
512 846
512 847
512 848
512 849
512 850
512 851
512 852
512 853
512 854
512 855
512 856
512 857
512 858
512 859
512 860
512 861
512 862
512 863
512 864
512 865
512 866
512 867
512 868
512 869
512 870
512 871
512 872
512 873
512 874
512 875
512 876
512 877
512 878
512 879
512 880
512 881
512 882
512 883
512 884
512 885
512 886
512 887
512 888
512 889
512 890
512 891
512 892
512 893
512 894
512 895
512 896
512 897
5

C:\Users\kyled\AppData\Local\Temp\ipykernel_10920\4137390217.py:334: UserWarning: Warning: converting a masked element to nan.
  stp_fl = np.asarray(stp_fl)
C:\Users\kyled\AppData\Local\Temp\ipykernel_10920\4137390217.py:335: UserWarning: Warning: converting a masked element to nan.
  stp_500 = np.asarray(stp_500)
